# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [5]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile)
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [37]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [38]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [39]:
try:
    session.set_keyspace('event_data')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1
#### PRIMARY KEY defined as sessionId and itemInSession to provide a definition of a unique row within this subset of data.

In [52]:
query = "CREATE TABLE IF NOT EXISTS artist_title_length_of_song "
query = query + "( sessionId int, itemInSession int, artist text, song text, length FLOAT, \
                 PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [53]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO artist_title_length_of_song (sessionId,  itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

#### Do a SELECT to verify that the data have been inserted into each table

In [54]:
query = "SELECT artist, song, length FROM artist_title_length_of_song WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2
#### PRIMARY KEY defined as 'userId' and 'sessionId' to provide a definition of a unique row within this subset of data.
#### A clustering column has been defined with 'itemInSession' to provide an ascending sort order in the returned dataset

In [43]:
query2 = "CREATE TABLE IF NOT EXISTS artist_and_songs_played_by_named_users "
query2 = query2 + "(userId int, sessionId int, itemInSession int, song text, artist text, \
                 firstName text, lastName text, PRIMARY KEY ((userId, sessionId) , itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

#### There we put data from csv into the database

In [45]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO artist_and_songs_played_by_named_users (userId, sessionId, itemInSession, song, artist, firstName, lastName)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

#### There we make the query which fetches artist, song and userID of related row.

In [46]:
query2 = "SELECT artist, song, userId FROM artist_and_songs_played_by_named_users WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", userid=10)
Row(artist='Three Drives', song='Greece 2000', userid=10)
Row(artist='Sebastien Tellier', song='Kilometer', userid=10)
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', userid=10)


## Query 3
#### PRIMARY KEY defined as the where item 'songs' and 'userId' to create a definition of a unique row

In [47]:
query3 = "CREATE TABLE IF NOT EXISTS users_by_song "
query3 = query3 + "(song text, userId int,firstName text, lastName text, PRIMARY KEY (song ,userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

#### There we put data from csv into the database

In [48]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query3 = "INSERT INTO users_by_song (song, userId , firstName, lastName)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

#### There we make the query which fetches firstName and lastName at rows where song name is All hands against his own.

In [49]:
query3 = "SELECT firstName, lastName FROM users_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)
                    

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [55]:
queries = ["DROP table artist_title_length_of_song" ,"DROP table artist_and_songs_played_by_named_users" , "DROP table users_by_song"]
try:
    for query in queries: 
        rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [56]:
session.shutdown()
cluster.shutdown()